In [1]:
import pandas as pd
import numpy as np
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
granularity = ['traffic_source']

## Categories

### Rates

In [4]:
pd.read_hdf('../generated/cat_rates.hdf5.csv.5', stop=10).head()

,category_id,platform,traffic_source,geo,cat_viewed
0,1000,1,1,-1,20.822754
1,1000,1,1,0,239.240352
2,1000,1,1,1,3.141090
3,1000,1,1,5,0.032317
4,1000,1,1,8,0.128752


In [5]:
rates = pd.read_hdf('../generated/cat_rates.hdf5.csv.5')

if len(granularity)>0:
    rates = pd.DataFrame({'weight':(rates.groupby(['category_id']+granularity)['cat_viewed'].sum())}).reset_index()
    rates_granularity = pd.DataFrame({'weight_g':rates.groupby(granularity)['weight'].sum()}).reset_index()
    rates = rates.merge(rates_granularity, on=granularity)
    rates['weight'] = rates['weight'] / rates['weight_g']  
    rates.drop(['weight_g'], axis=1, inplace=True)
                          
rates['weight_sqrt'] = np.sqrt(rates['weight'])
rates.head()

,category_id,traffic_source,weight,weight_sqrt
0,1000,1,0.000792,0.028146
1,1100,1,0.017732,0.133163
2,1202,1,0.000456,0.021358
3,1203,1,0.004093,0.063973
4,1204,1,0.001038,0.032223


In [6]:
rates.count()

category_id       274
traffic_source    274
weight            274
weight_sqrt       274
dtype: int64

In [7]:
sum(rates['weight'])

2.9999999999999996

### Docs pairs

In [8]:
page_addoc_dist = pd.read_csv("../generated/final/events_clicks_CTR_addocs_train.csv", usecols=['document_id', 'addoc_id']+granularity)
page_addoc_dist.count()

document_id       87141731
traffic_source    87141731
addoc_id          87141731
dtype: int64

In [9]:
page_addoc_dist = pd.DataFrame({'size':page_addoc_dist.groupby(['document_id', 'addoc_id']+granularity).size()}).reset_index().drop(['size'], axis=1)
page_addoc_dist.count()

document_id       31484821
addoc_id          31484821
traffic_source    31484821
dtype: int64

In [10]:
page_addoc_dist.to_csv("../generated/page_addoc_dist.1.csv", index=False) #traffic_source

### Docs categories and norms

In [11]:
len(page_addoc_dist.document_id.unique()), len(page_addoc_dist.addoc_id.unique())

(756510, 164476)

In [12]:
docs_used = pd.DataFrame({'document_id':page_addoc_dist.document_id.append(page_addoc_dist.addoc_id).unique()})
docs_used.count()

document_id    912081
dtype: int64

In [13]:
del page_addoc_dist

In [14]:
docs_cats = pd.read_csv("../download/documents_categories.csv")
docs_cats = docs_cats.merge(docs_used, on='document_id')
docs_cats.head()

,document_id,category_id,confidence_level
0,1595802,1611,0.92
1,1595802,1610,0.07
2,1524246,1807,0.92
3,1524246,1608,0.07
4,1617787,1807,0.92


In [15]:
print len(docs_cats)

1757679


In [16]:
docs_cats = docs_cats.merge(rates, on='category_id', how='left').fillna(0)
docs_cats['value']=docs_cats['confidence_level']*docs_cats['weight_sqrt']
docs_cats['value2']=np.square(docs_cats['value'])
docs_cats.drop(['confidence_level','weight','weight_sqrt'], axis=1, inplace=True)
docs_cats.head()

,document_id,category_id,traffic_source,value,value2
0,1595802,1611,1.0,0.083634,0.006995
1,1595802,1611,2.0,0.071967,0.005179
2,1595802,1611,3.0,0.113503,0.012883
3,1595802,1610,1.0,0.007290,0.000053
4,1595802,1610,2.0,0.005852,0.000034


In [17]:
print len(docs_cats)

5272995


In [18]:
docs_norms2 = pd.DataFrame({'norm2':docs_cats.groupby(['document_id'] + granularity)['value2'].sum()}).reset_index()
docs_cats.drop(['value2'], axis=1, inplace=True)
docs_norms2.head()

,document_id,traffic_source,norm2
0,1,1.0,0.006166
1,1,2.0,0.003093
2,1,3.0,0.006457
3,2,1.0,0.011808
4,2,2.0,0.005673


### Distances

In [19]:
input_file = "../generated/page_addoc_dist.1.csv"
output_file = "../generated/final/page_addoc_categories_t_dist.csv"
chunksize=10**6
how='left'

In [20]:
first_time = True
size = 0
i = 0
    
f = open(output_file, 'w')

for page_addoc_dist in pd.read_csv(input_file, chunksize=chunksize):

    page_addoc_diffs = page_addoc_dist.merge(docs_cats, left_on=['addoc_id']+granularity, right_on=['document_id']+granularity)
    page_addoc_diffs.rename(columns={'document_id_x':'document_id'}, inplace=True)
    page_addoc_diffs.drop(['document_id_y'], axis=1, inplace=True)
    page_addoc_diffs = page_addoc_diffs.merge(docs_cats, on=['document_id', 'category_id']+granularity)
    page_addoc_diffs['value_2xy'] = 2*page_addoc_diffs['value_x']*page_addoc_diffs['value_y']
    page_addoc_diffs.drop(['category_id', 'value_x', 'value_y'], axis=1, inplace=True)
    page_addoc_diffs = pd.DataFrame({'value_2xy':page_addoc_diffs.groupby(['document_id', 'addoc_id']+granularity)['value_2xy'].sum()}).reset_index()
    
    page_addoc_dist = page_addoc_dist.merge(docs_norms2, on=['document_id']+granularity)
    page_addoc_dist = page_addoc_dist.merge(docs_norms2, left_on=['addoc_id']+granularity, right_on=['document_id']+granularity)
    page_addoc_dist.rename(columns={'document_id_x':'document_id'}, inplace=True)
    page_addoc_dist.drop(['document_id_y'], axis=1, inplace=True)
    page_addoc_dist = page_addoc_dist.merge(page_addoc_diffs, on=['document_id', 'addoc_id']+granularity, how='left').fillna(0)
    page_addoc_dist['dist_cats'] = np.sqrt( page_addoc_dist['norm2_x'] + page_addoc_dist['norm2_y'] - page_addoc_dist['value_2xy'] )
    page_addoc_dist.drop(['norm2_x', 'norm2_y', 'value_2xy'], axis=1, inplace=True)

    if first_time:
        page_addoc_dist.to_csv(f, index = False, header=True)
        f.close()
        f = open(output_file, 'a')
        first_time = False
    else:
        page_addoc_dist.to_csv(f, index = False, header=False)
    
    size+=chunksize
    i+=1
    print i, size
            
print 'Done'
    
f.close()

1 1000000
2 2000000
3 3000000
4 4000000
5 5000000
6 6000000
7 7000000
8 8000000
9 9000000
10 10000000
11 11000000
12 12000000
13 13000000
14 14000000
15 15000000
16 16000000
17 17000000
18 18000000
19 19000000
20 20000000
21 21000000
22 22000000
23 23000000
24 24000000
25 25000000
26 26000000
27 27000000
28 28000000
29 29000000
30 30000000
31 31000000
32 32000000
Done


In [21]:
pd.read_csv("../generated/final/page_addoc_categories_t_dist.csv", nrows=5).head()

,document_id,addoc_id,traffic_source,dist_cats
0,2,1489042,1.0,0.163817
1,452,1489042,1.0,0.138173
2,515,1489042,1.0,0.137168
3,7616,1489042,1.0,0.126194
4,7794,1489042,1.0,0.124072
